In [15]:
import requests,json
import re
from bs4 import BeautifulSoup 

In [49]:
list_url = "https://y.qq.com/n/yqq/playlist/7628476949.html"
headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    }
response = requests.get(list_url, headers=headers)
html_txt = response.text 
# 原本尝试抓json的，毕竟json处理起来easy，但是抓不到
# json的地址不是上面的那个url，抓到的json 显示 invalid refere
# 问了HFF，他说应该是 异步请求，防跨域的
# 所以打算直接 正则匹配，提取 歌名 - 歌手
# html_txt

In [60]:
soup = BeautifulSoup(html_txt, 'html.parser')
# soup.select('.songlist__list')
# soup.find_all('ul', class_='songlist__list') # 与上面效果等同

### 获取歌曲名字

In [61]:
soup.find_all('span', class_='songlist__songname_txt') # 注意是class_，不是class，因为class是python的关键字

[<span class="songlist__songname_txt"><a href="javascript:;" title="Youth (Acoustic)">Youth (Acoustic)</a></span>,
 <span class="songlist__songname_txt"><a href="javascript:;" title="Hall of Fame">Hall of Fame</a></span>,
 <span class="songlist__songname_txt"><a href="javascript:;" title="年少有为">年少有为</a></span>,
 <span class="songlist__songname_txt"><a href="javascript:;" title="Fun">Fun</a></span>,
 <span class="songlist__songname_txt"><a href="javascript:;" title="北京东路的日子">北京东路的日子</a></span>]

In [59]:
song_name_list = [
    song.string for song in soup.find_all('span', class_='songlist__songname_txt')
] # string属性就可以获取指定标签的文本属性
song_name_list

['Youth (Acoustic)', 'Hall of Fame', '年少有为', 'Fun', '北京东路的日子']

### 获取歌手

In [70]:
soup.find_all('a', class_='singer_name')

[<a class="singer_name" href="javascript:;" title="Troye Sivan">Troye Sivan</a>,
 <a class="singer_name" href="javascript:;" title="The Script">The Script</a>,
 <a class="singer_name" href="javascript:;" title="will.i.am">will.i.am</a>,
 <a class="singer_name" href="javascript:;" title="李荣浩">李荣浩</a>,
 <a class="singer_name" href="javascript:;" title="Troye Sivan">Troye Sivan</a>,
 <a class="singer_name" href="javascript:;" title="华语群星">华语群星</a>]

这种方法有一个问题，比如这里 Hall of fame 这首歌有多个歌手，如果依次对应上面的 歌曲名称列表，就会对应不上   
扩大标签级别（找上一层标签）

In [71]:
soup.find_all('div', class_='songlist__artist')

[<div class="songlist__artist">
 <a class="singer_name" href="javascript:;" title="Troye Sivan">Troye Sivan</a>
 </div>,
 <div class="songlist__artist">
 <a class="singer_name" href="javascript:;" title="The Script">The Script</a>
                                 
                                 /
 					<a class="singer_name" href="javascript:;" title="will.i.am">will.i.am</a>
 </div>,
 <div class="songlist__artist">
 <a class="singer_name" href="javascript:;" title="李荣浩">李荣浩</a>
 </div>,
 <div class="songlist__artist">
 <a class="singer_name" href="javascript:;" title="Troye Sivan">Troye Sivan</a>
 </div>,
 <div class="songlist__artist">
 <a class="singer_name" href="javascript:;" title="华语群星">华语群星</a>
 </div>]

In [73]:
soup.find_all('div', class_='songlist__artist')[0]

<div class="songlist__artist">
<a class="singer_name" href="javascript:;" title="Troye Sivan">Troye Sivan</a>
</div>

In [74]:
soup.find_all('div', class_='songlist__artist')[0].string # 可以看到为空，因为这个标签里面是一个 <a>标签，没有文本

In [77]:
soup.find_all('div', class_='songlist__artist')[0].a

<a class="singer_name" href="javascript:;" title="Troye Sivan">Troye Sivan</a>

In [79]:
soup.find_all('div', class_='songlist__artist')[0].a.string

'Troye Sivan'

In [81]:
soup.find_all('div', class_='songlist__artist')[1]

<div class="songlist__artist">
<a class="singer_name" href="javascript:;" title="The Script">The Script</a>
                                
                                /
					<a class="singer_name" href="javascript:;" title="will.i.am">will.i.am</a>
</div>

In [80]:
soup.find_all('div', class_='songlist__artist')[1].a # 这样会丢失多余的<a>，只提取了一个

<a class="singer_name" href="javascript:;" title="The Script">The Script</a>

那么有2种方法来提取对应歌手

In [ ]:
for singer